In [6]:
import os
import glob
import random
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf

In [2]:
tf.__version__

'2.14.0'

In [4]:
_ROOT = "E:\Projects\image_classification_dog_cat"

In [5]:
RAW_DATA_ROOT = os.path.join(_ROOT,"data\\raw\\training_set")

In [7]:
os.listdir(RAW_DATA_ROOT)

['cats', 'dogs']

In [34]:
images_list = []
for root, dirs, files in os.walk(RAW_DATA_ROOT):
    for f in files:
        if f.endswith('.jpg'):
            images_list.append(os.path.join(root, f))

print(len(images_list))

8005


In [89]:
demo_img = cv2.imread(images_list[random.randint(1, 1000)])
print(demo_img.shape)

(438, 499, 3)


In [100]:
demo_img.shape[:-1]

(418, 499)

In [110]:
img_size_x = []
img_size_y = []
img_size = []
for img in images_list:
    demo_img = cv2.imread(img)
    img_size.append(demo_img[:-1])
    img_size_x.append(demo_img.shape[0])
    img_size_y.append(demo_img.shape[1])

In [111]:
len(img_size_x), len(img_size_y)

(8005, 8005)

In [112]:
len(img_size)

8005

In [113]:
img_size_x[0]

280

In [114]:
np.median(img_size_x), np.median(img_size_y)

(374.0, 445.0)

In [116]:
np.mean(img_size_x), np.mean(img_size_y)

(360.494440974391, 403.7998750780762)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(
    DATA_ROOT,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary')

val_set = val_datagen.flow_from_directory(
    DATA_ROOT,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary')

In [118]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [119]:

# Initialising the CNN
model = Sequential()

# Step 1 - Convolution
model.add(Conv2D(32, (3, 3), input_shape = (256, 256, 3), activation = 'relu'))

# Step 2 - Pooling
model.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer




# Step 3 - Flattening
model.add(Flatten())

# Step 4 - Full connection
model.add(Dense(units = 128, activation = 'relu'))

model.add(Dense(units = 1, activation = 'sigmoid'))

model.summary()
# Compiling the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 516128)            0         
                                                                 
 dense (Dense)               (None, 128)               66064512  
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 66065537 (252.02 MB)
Trainable params: 66065537 (252.02 MB)
Non-trainable params: 0 (0.00 Byte)
____________

In [ ]:
history = model.fit_generator(
    training_set,
    steps_per_epoch=8000,
    epochs=1,
    validation_data=val_set,
    validation_steps=2000)